# The Capability Architectural Pattern

## The Capability Pattern

In [ ]:
########################################################################
# The Capability pattern allows us to modify multiple aspects of the
# agent loop while keeping the core logic clean and maintainable.
#
# The idea behind the Capability pattern is to encapsulate specific
# adaptations of the agent loop inside of a class. This class can be
# plugged in to modify the behavior of the agent loop without
# modifying the loop code itself. Agent’s that need more specialized
# agent loop behavior can be composed by adding capabilities to the
# agent. The Capability has a lifecycle that begins when the agent
# loop is about to start and ends when the agent loop is about to
# terminate. A Capability might open a database connection, log
# prompts being sent to the LLM, or add metadata to the agent’s
# responses.
########################################################################

def run(self, user_input: str, memory=None, action_context_props=None):

    ... existing code ...
    
    # Initialize capabilities
    for capability in self.capabilities:
        capability.init(self, action_context)
        
    while True:
        # Start of loop capabilities
        can_start_loop = reduce(lambda a, c: c.start_agent_loop(self, action_context),
                              self.capabilities, False)

        ... existing code ...
        
        # Construct prompt with capability modifications
        prompt = reduce(lambda p, c: c.process_prompt(self, action_context, p),
                      self.capabilities, base_prompt)

        ... existing code ...
        
        # Process response with capabilities
        response = reduce(lambda r, c: c.process_response(self, action_context, r),
                        self.capabilities, response)

        ... existing code ...
        
        # Process action with capabilities
        action = reduce(lambda a, c: c.process_action(self, action_context, a),
                      self.capabilities, action)
        
        ... existing code ...
        
        # Process result with capabilities
        result = reduce(lambda r, c: c.process_result(self, action_context, response,
                                                     action_def, action, r),
                       self.capabilities, result)

        ... existing code ...
        
        # End of loop capabilities
        for capability in self.capabilities:
            capability.end_agent_loop(self, action_context)

## Understanding the Capability Class

In [ ]:
# A Capability can interact with the agent loop at multiple points.
# Think of these interaction points like hooks or lifecycle events in
# a web framework - they give us specific moments where we can modify
# or enhance the agent’s behavior.

class Capability:
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def init(self, agent, action_context: ActionContext) -> dict:
        """Called once when the agent starts running."""
        pass

    def start_agent_loop(self, agent, action_context: ActionContext) -> bool:
        """Called at the start of each iteration through the agent loop."""
        return True

    def process_prompt(self, agent, action_context: ActionContext, 
                      prompt: Prompt) -> Prompt:
        """Called right before the prompt is sent to the LLM."""
        return prompt

    def process_response(self, agent, action_context: ActionContext, 
                        response: str) -> str:
        """Called after getting a response from the LLM."""
        return response

    def process_action(self, agent, action_context: ActionContext, 
                      action: dict) -> dict:
        """Called after parsing the response into an action."""
        return action

    def process_result(self, agent, action_context: ActionContext,
                      response: str, action_def: Action,
                      action: dict, result: any) -> any:
        """Called after executing the action."""
        return result

    def process_new_memories(self, agent, action_context: ActionContext,
                           memory: Memory, response, result,
                           memories: List[dict]) -> List[dict]:
        """Called when new memories are being added."""
        return memories

    def end_agent_loop(self, agent, action_context: ActionContext):
        """Called at the end of each iteration through the agent loop."""
        pass

    def should_terminate(self, agent, action_context: ActionContext,
                        response: str) -> bool:
        """Called to check if the agent should stop running."""
        return False

    def terminate(self, agent, action_context: ActionContext) -> dict:
        """Called when the agent is shutting down."""
        pass

In [ ]:
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment,
                 capabilities: List[Capability] = [],
                 max_iterations: int = 10,
                 max_duration_seconds: int = 180):
        """
        Initialize an agent with its core GAME components and capabilities.
        
        Goals, Actions, Memory, and Environment (GAME) form the core of the agent,
        while capabilities provide ways to extend and modify the agent's behavior.
        
        Args:
            goals: What the agent aims to achieve
            agent_language: How the agent formats and parses LLM interactions
            action_registry: Available tools the agent can use
            generate_response: Function to call the LLM
            environment: Manages tool execution and results
            capabilities: List of capabilities that extend agent behavior
            max_iterations: Maximum number of action loops
            max_duration_seconds: Maximum runtime in seconds
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment
        self.capabilities = capabilities or []
        self.max_iterations = max_iterations
        self.max_duration_seconds = max_duration_seconds

In [ ]:
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment,
                 capabilities: List[Capability] = [],
                 max_iterations: int = 10,
                 max_duration_seconds: int = 180):
        """
        Initialize an agent with its core GAME components and capabilities.
        
        Goals, Actions, Memory, and Environment (GAME) form the core of the agent,
        while capabilities provide ways to extend and modify the agent's behavior.
        
        Args:
            goals: What the agent aims to achieve
            agent_language: How the agent formats and parses LLM interactions
            action_registry: Available tools the agent can use
            generate_response: Function to call the LLM
            environment: Manages tool execution and results
            capabilities: List of capabilities that extend agent behavior
            max_iterations: Maximum number of action loops
            max_duration_seconds: Maximum runtime in seconds
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment
        self.capabilities = capabilities or []
        self.max_iterations = max_iterations
        self.max_duration_seconds = max_duration_seconds

## Implementing Time Awareness

In [ ]:
########################################################################
# The TimeAwareCapability needs to inform the agent about the current
# time and ensure this information persists throughout its
# decision-making process:
########################################################################

from datetime import datetime
from zoneinfo import ZoneInfo

class TimeAwareCapability(Capability):
    def __init__(self):
        super().__init__(
            name="Time Awareness",
            description="Allows the agent to be aware of time"
        )
        
    def init(self, agent, action_context: ActionContext) -> dict:
        """Set up time awareness at the start of agent execution."""
        # Get timezone from context or use default
        time_zone_name = action_context.get("time_zone", "America/Chicago")
        timezone = ZoneInfo(time_zone_name)
        
        # Get current time in specified timezone
        current_time = datetime.now(timezone)
        
        # Format time in both machine and human-readable formats
        iso_time = current_time.strftime("%Y-%m-%dT%H:%M:%S%z")
        human_time = current_time.strftime("%H:%M %A, %B %d, %Y")
        
        # Store time information in memory
        memory = action_context.get_memory()
        memory.add_memory({
            "type": "system",
            "content": f"""Right now, it is {human_time} (ISO: {iso_time}).
            You are in the {time_zone_name} timezone.
            Please consider the day/time, if relevant, when responding."""
        })
        
    def process_prompt(self, agent, action_context: ActionContext, 
                      prompt: Prompt) -> Prompt:
        """Update time information in each prompt."""
        time_zone_name = action_context.get("time_zone", "America/Chicago")
        current_time = datetime.now(ZoneInfo(time_zone_name))
        
        # Add current time to system message
        system_msg = (f"Current time: "
                     f"{current_time.strftime('%H:%M %A, %B %d, %Y')} "
                     f"({time_zone_name})\n\n")
        
        # Add to existing system message or create new one
        messages = prompt.messages
        if messages and messages[0]["role"] == "system":
            messages[0]["content"] = system_msg + messages[0]["content"]
        else:
            messages.insert(0, {
                "role": "system",
                "content": system_msg
            })
            
        return Prompt(messages=messages)

In [ ]:
agent = Agent(
    goals=[Goal(name="task", description="Complete the assigned task")],
    agent_language=JSONAgentLanguage(),
    action_registry=registry,
    generate_response=llm.generate,
    environment=PythonEnvironment(),
    capabilities=[
        TimeAwareCapability()
    ]
)

In [ ]:
# Example conversation
agent.run("Schedule a team meeting for today")

# Agent response might include:
"Since it's already 5:30 PM on Friday, I recommend scheduling the meeting 
for Monday morning instead. Would you like me to look for available times 
on Monday?"

# Extending the Time Awareness Capability

## Extending the Time Awareness Capability

In [ ]:
class EnhancedTimeAwareCapability(TimeAwareCapability):
    def process_action(self, agent, action_context: ActionContext, 
                      action: dict) -> dict:
        """Add timing information to action results."""
        # Add execution time to action metadata
        action["execution_time"] = datetime.now(
            ZoneInfo(action_context.get("time_zone", "America/Chicago"))
        ).isoformat()
        return action
        
    def process_result(self, agent, action_context: ActionContext,
                      response: str, action_def: Action,
                      action: dict, result: any) -> any:
        """Add duration information to results."""
        if isinstance(result, dict):
            result["action_duration"] = (
                datetime.now(ZoneInfo(action_context.get("time_zone"))) -
                datetime.fromisoformat(action["execution_time"])
            ).total_seconds()
        return result

## Plan First: A New Capability

In [ ]:
########################################################################
# Here’s how we’ll make our agent plan before acting:
# 
# When the agent first starts, we’ll prompt it to create a detailed plan
# We’ll store this plan in the agent’s memory
# The agent will refer back to this plan throughout its execution
# Let’s implement this using a capability:
########################################################################

class PlanFirstCapability(Capability):
    def __init__(self, plan_memory_type="system", track_progress=False):
        super().__init__(
            name="Plan First Capability",
            description="The Agent will always create a plan and add it to memory"
        )
        self.plan_memory_type = plan_memory_type
        self.first_call = True
        self.track_progress = track_progress

    def init(self, agent, action_context):
        if self.first_call:
            self.first_call = False
            plan = create_plan(
                action_context=action_context,
                memory=action_context.get_memory(),
                action_registry=action_context.get_action_registry()
            )

            action_context.get_memory().add_memory({
                "type": self.plan_memory_type,
                "content": "You must follow these instructions carefully to complete the task:\n" + plan
            })

# The key is in our create_plan function, which guides the LLM through 
# structured thinking:

@register_tool(tags=["planning"])
def create_plan(action_context: ActionContext,
                memory: Memory,
                action_registry: ActionRegistry) -> str:
   """Create a detailed execution plan based on the task and available tools."""

   # Get tool descriptions for the prompt
   tool_descriptions = "\n".join(
      f"- {action.name}: {action.description}"
      for action in action_registry.get_actions()
   )

   # Get relevant memory content
   memory_content = "\n".join(
      f"{m['type']}: {m['content']}"
      for m in _memory.items
      if m['type'] in ['user', 'system']
   )

   # Construct the prompt as a string
   prompt = f"""Given the task in memory and the available tools, create a detailed plan.
Think through this step by step:

1. First, identify the key components of the task
2. Consider what tools you have available
3. Break down the task into logical steps
4. For each step, specify:
   - What needs to be done
   - What tool(s) will be used
   - What information is needed
   - What the expected outcome is

Write your plan in clear, numbered steps. Each step should be specific and actionable.

Available tools:
{tool_descriptions}

Task context from memory:
{memory_content}

Create a plan that accomplishes this task effectively."""

   return prompt_llm(action_context=action_context, prompt=prompt)

agent = Agent(
    goals=[
        Goal(name="analysis",
             description="Analyze sales data and create a report")
    ],
    capabilities=[
        PlanFirstCapability(track_progress=True)
    ],
    # ... other agent configuration
)

result = agent.run("Analyze our Q4 sales data and create a report")

# Tracking Progress in the Agent Loop

## Tracking Progress

In [ ]:
########################################################################
# In complex tasks, agents need to periodically step back and assess
# their progress. Let’s build a capability that adds reflection and
# progress tracking to the end of each agent loop iteration, allowing
# the agent to understand what it just did and plan its next steps
# more effectively.
#
# To achieve this, we introduce a track_progress function that
# evaluates the agent’s current state after each action. This function
# operates similarly to our planning function but shifts the focus to
# assessment rather than forward planning. By analyzing available
# tools, memory context, and completed steps, the agent can identify
# what has been accomplished, any blockers encountered, and what
# should happen next. This ensures the agent maintains awareness of
# its trajectory rather than blindly executing actions without
# reflection.
#
# By integrating track_progress at the end of each loop iteration, we
# enable the agent to continuously refine its strategy. Instead of
# relying solely on a predefined plan, the agent dynamically adapts
# based on real-time feedback. This aligns with how human
# problem-solving works—we plan, act, evaluate, and adjust. With this
# addition, our agent becomes more resilient and capable, recognizing
# obstacles early and making course corrections as needed, and
# potentially leading to more efficient and intelligent execution of
# complex workflows.
########################################################################

@register_tool(tags=["prompts"])
def track_progress(action_context: ActionContext,
                   _memory: Memory,
                   action_registry: ActionRegistry) -> str:
    """Generate a progress report based on the current task, available tools, and memory context."""

    # Get tool descriptions for the prompt
    tool_descriptions = "\n".join(
        f"- {action.name}: {action.description}"
        for action in action_registry.get_actions()
    )

    # Get relevant memory content
    memory_content = "\n".join(
        f"{m['type']}: {m['content']}"
        for m in _memory.items
        if m['type'] in ['user', 'system']
    )

    # Construct the prompt as a string
    prompt = f"""Given the current task and available tools, generate a progress report.
Think through this step by step:

1. Identify the key components of the task and the intended outcome.
2. Assess the progress made so far based on available information.
3. Identify any blockers or issues preventing completion.
4. Suggest the next steps to move forward efficiently.
5. Recommend any tool usage that might help complete the task.

Write your progress report in clear, structured points.

Available tools:
{tool_descriptions}

Task context from memory:
{memory_content}

Provide a well-organized report on the current progress and next steps."""

    return prompt_llm(action_context=action_context, prompt=prompt)

## Building a Progress Tracking Capability

In [ ]:
########################################################################
# Now that we have a track_progress function, let’s turn it into a
# Capability that we can add to our agent. This capability adds
# overhead and slows down our agent, since it will add one prompt per
# agent loop, but it can help with complex tasks where tracking
# progress is essential. Here’s how we can implement a capability that
# tracks progress after each action:
########################################################################

class ProgressTrackingCapability(Capability):
    def __init__(self, memory_type="system", track_frequency=1):
        super().__init__(
            name="Progress Tracking",
            description="Tracks progress and enables reflection after actions"
        )
        self.memory_type = memory_type
        self.track_frequency = track_frequency
        self.iteration_count = 0

    def end_agent_loop(self, agent, action_context: ActionContext):
        """Generate and store progress report at the end of each iteration."""
        self.iteration_count += 1
        
        # Only track progress on specified iterations
        if self.iteration_count % self.track_frequency != 0:
            return
            
        # Get the memory and action registry from context
        memory = action_context.get_memory()
        action_registry = action_context.get_action_registry()
        
        # Generate progress report
        progress_report = track_progress(
            action_context=action_context,
            _memory=memory,
            action_registry=action_registry
        )
        
        # Add the progress report to memory
        memory.add_memory({
            "type": self.memory_type,
            "content": f"Progress Report (Iteration {self.iteration_count}):\n{progress_report}"
        })

In [ ]:
# Create an agent with progress tracking
agent = Agent(
    goals=[
        Goal(
            name="data_processing",
            description="Process and analyze customer feedback data"
        )
    ],
    capabilities=[
        ProgressTrackingCapability(track_frequency=2)  # Track every 2nd iteration
    ],
    # ... other agent configuration
)

# Example execution flow
memory = agent.run("Analyze customer feedback from Q4 and identify top issues")